In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast, TFBertModel
import numpy as np
from transformers import AdamW
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Dense,Flatten,Softmax,Layer,Input
from tensorflow.keras import Sequential,Model
import tensorflow as tf
# specify GPU
# device = torch.device("cuda")

In [2]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.1 MB/s eta 0:00:00


In [10]:
title=pd.read_csv('titles.csv')
# desc=pd.read_excel('descriptions.xls')
title.replace('Good',0,inplace=True)
title.replace('Average',1,inplace=True)
title.replace('Bad',2,inplace=True)

print(title.count())
print(title.groupby('Category').count())
X=title.Titles.values
Y=title.Category.values

Titles      418
Category    418
dtype: int64
          Titles
Category        
0            163
1            100
2            155


In [11]:
X_train,X_temp,Y_train,Y_temp=train_test_split(X,Y,test_size=0.3,random_state=10)
X_test,X_val,Y_test,Y_val=train_test_split(X_temp,Y_temp,test_size=0.5,random_state=10)
print(X_train.shape,Y_train.shape)
print(X_val.shape,Y_val.shape)
print(X_test.shape,Y_test.shape)

temp_Y=[]
for y in Y_train:
    arr=[0,0,0]
    arr[y]=1
    temp_Y.append(arr)

Y_train=temp_Y

temp_Y=[]
for y in Y_val:
    arr=[0,0,0]
    arr[y]=1
    temp_Y.append(arr)

Y_val=temp_Y

temp_Y=[]
for y in Y_test:
    arr=[0,0,0]
    arr[y]=1
    temp_Y.append(arr)

Y_test=temp_Y

print(Y_train)

(292,) (292,)
(63,) (63,)
(63,) (63,)
[[1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 

In [6]:
# import BERT-base pretrained model
# bert = AutoModel.from_pretrained('bert-base-uncased')
bert = TFBertModel.from_pretrained('bert-base-uncased')
# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [12]:
X_tokenize=tokenizer(X_train.tolist(), max_length=25, pad_to_max_length=True, truncation=True, return_tensors="tf")
X_train={
    'input_ids':np.array(X_tokenize['input_ids']),
    'token_type_ids':np.array(X_tokenize['token_type_ids']),
    'attention_mask':np.array(X_tokenize['attention_mask'])
}
X_tokenize=tokenizer(X_val.tolist(), max_length=25, pad_to_max_length=25, truncation=True, return_tensors="tf")
X_val={
    'input_ids':np.array(X_tokenize['input_ids']),
    'token_type_ids':np.array(X_tokenize['token_type_ids']),
    'attention_mask':np.array(X_tokenize['attention_mask'])
}
X_tokenize=tokenizer(X_test.tolist(), max_length=25, pad_to_max_length=25, truncation=True, return_tensors="tf")
X_test={
    'input_ids':np.array(X_tokenize['input_ids']),
    'token_type_ids':np.array(X_tokenize['token_type_ids']),
    'attention_mask':np.array(X_tokenize['attention_mask'])
}

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [13]:
bert_train_output = bert([X_train['input_ids'],X_train['token_type_ids'],X_train['attention_mask']])
bert_val_output = bert([X_val['input_ids'],X_val['token_type_ids'],X_val['attention_mask']])
bert_test_output = bert([X_test['input_ids'],X_test['token_type_ids'],X_test['attention_mask']])

In [27]:
print(bert_train_output)

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(292, 25, 768), dtype=float32, numpy=
array([[[ 0.08285062, -0.29926285,  0.36248383, ...,  0.05021085,
         -0.05284922,  0.18111694],
        [ 0.4883227 , -0.17124465,  1.200596  , ..., -0.15674   ,
         -0.02089807,  0.28013203],
        [-0.33757102,  0.39076942,  0.5809097 , ..., -0.7597159 ,
          0.42583558, -0.47260708],
        ...,
        [ 0.2769272 , -0.35702875,  0.7640461 , ..., -0.6724845 ,
          0.19084609, -0.8067247 ],
        [ 0.28225237, -0.3058215 ,  0.67706513, ..., -0.6534976 ,
          0.16509372, -0.897594  ],
        [ 0.16474187, -0.05629085,  0.53953433, ..., -0.68257415,
          0.07173171, -1.1246183 ]],

       [[ 0.09799346, -0.18221867,  0.43842286, ..., -0.28576753,
          0.09227575,  0.16932659],
        [-0.0216775 , -0.1564136 ,  0.9285548 , ...,  0.02038541,
          0.11338776, -0.0502399 ],
        [ 0.10590921,  0.10755712,  0.8654626 , 

In [16]:
print(bert_train_output.last_hidden_state.shape)
print(bert_val_output.last_hidden_state.shape)
print(bert_test_output.last_hidden_state.shape)

(292, 25, 768)
(63, 25, 768)
(63, 25, 768)


In [19]:
model = Sequential([
    Input(shape=(25,768)),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(3,activation='softmax'),
])

In [21]:
model.compile(optimizer="Adam", loss="CategoricalCrossentropy", metrics=["accuracy"])

In [29]:
model.fit(np.array(bert_train_output.last_hidden_state).tolist(),Y_train,validation_data=(np.array(bert_val_output.last_hidden_state).tolist(),Y_val),epochs=11)

Epoch 1/11
10/10 [==============================] - 8s 823ms/step - loss: 1.2830 - accuracy: 0.8733 - val_loss: 0.7623 - val_accuracy: 0.8889
Epoch 2/11
10/10 [==============================] - 2s 180ms/step - loss: 0.1549 - accuracy: 0.9623 - val_loss: 0.4247 - val_accuracy: 0.9206
Epoch 3/11
10/10 [==============================] - 2s 178ms/step - loss: 0.0936 - accuracy: 0.9760 - val_loss: 0.1694 - val_accuracy: 0.9683
Epoch 4/11
10/10 [==============================] - 2s 215ms/step - loss: 0.0669 - accuracy: 0.9863 - val_loss: 0.0596 - val_accuracy: 0.9683
Epoch 5/11
10/10 [==============================] - 3s 297ms/step - loss: 0.0061 - accuracy: 0.9966 - val_loss: 0.2407 - val_accuracy: 0.9524
Epoch 6/11
10/10 [==============================] - 2s 178ms/step - loss: 6.1804e-04 - accuracy: 1.0000 - val_loss: 0.0898 - val_accuracy: 0.9683
Epoch 7/11
10/10 [==============================] - 2s 175ms/step - loss: 3.9753e-05 - accuracy: 1.0000 - val_loss: 0.0395 - val_accuracy: 0.968

In [30]:
model.evaluate(np.array(bert_test_output.last_hidden_state).tolist(),Y_test)

2/2 [==============================] - 0s 25ms/step - loss: 1.0359e-05 - accuracy: 1.0000


[1.0359474799770396e-05, 1.0]

In [33]:
model.save('titles_bert.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
